In [42]:
import Pkg
Pkg.add("Turing")
Pkg.add("Flux")
Pkg.add("NCDatasets")
Pkg.add("TSVD")
Pkg.add("Statistics")
Pkg.add("Compat")
Pkg.add("LinearAlgebra")
Pkg.add("Glob")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Distributions")
Pkg.add("ProgressMeter")
Pkg.add("PyPlot")
Pkg.add("Random")
Pkg.add("StatsPlots")
Pkg.add("SpecialFunctions")
Pkg.add("ReverseDiff")
Pkg.add("BSON")
Pkg.add("TransformVariables")
Pkg.add("TransformedLogDensities")
Pkg.add("LogDensityProblems")
Pkg.add("LogDensityProblemsAD")
Pkg.add("DynamicHMC")
Pkg.add("Parameters")
Pkg.add("TVSD")
using Flux
using Statistics
using LinearAlgebra
using Compat
using Glob
using TSVD
using NCDatasets
using CSV
using DataFrames
using Distributions: Categorical, Dirichlet, Gamma, Beta
using ProgressMeter
using PyPlot
using Random
using SpecialFunctions: loggamma
using ReverseDiff
using BSON: @load

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Chan

LoadError: The following package names could not be resolved:
 * TVSD (not found in project, manifest or registry)
[36m   Suggestions:[39m Rela[0m[1mt[22mi[0m[1mv[22mi[0m[1ms[22mtic[0m[1mD[22mynamics Wa[0m[1mt[22merWa[0m[1mv[22me[0m[1ms[22m1[0m[1mD[22m Adap[0m[1mt[22mi[0m[1mv[22me[0m[1mS[22mparseGri[0m[1md[22ms

In [2]:
obs_file ="../data/observed_speeds/greenland_vel_mosaic250_v1_g9000m.nc"
d_obs = NCDataset(obs_file)
v_obs = d_obs["velsurf_mag"][:]
v_obs = nomissing(v_obs, 0.0);
idx = findall(v_obs .> 0)
Obs = v_obs[idx];

n_grid_points = size(idx)[1];

In [19]:
training_files = sort(glob("../tests/training_data/*.nc"))

nf = length(training_files)
d = NCDataset(training_files[1], "r")
v = d["velsurf_mag"]
nx, ny, nt = size(v)

Data = zeros(n_grid_points, nf * nt)
ids = zeros(Int64, nf)
@showprogress for (k, training_file) in enumerate(training_files)
    m_id = match(r"id_(.+?)_", training_file)
    ids[k] = parse(Int, m_id[1])
    d = NCDataset(training_file, "r")
    v = d["velsurf_mag"][:]
    v = nomissing(v, 0.0)
    Data[:, k] = v[idx]
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


## Read training samples

In [20]:
X_df = DataFrame(CSV.File("../data/samples/velocity_calibration_samples_50.csv"))
X_df = X_df[ [x in ids for x in X_df[!, :id]] ,:]
X = transpose(Matrix(X_df[!, 2:9]))
X_mean = mean(X, dims=2);
X_std = std(X, dims=2);
X_scaled = (X .- X_mean) ./ X_std;
X_train = X_scaled;
n_parameters, n_samples = size(X);

## Load the model

Should be a commmand line argument

That we have to define the struct again is not ok. How can we avoid this?

In [25]:
struct NNModel
    chain::Chain
    V_hat::AbstractArray
    F_mean::AbstractArray
end

function (m::NNModel)(x, add_mean=false)
    if add_mean
        return V_hat * m.chain(x) .+ F_mean
    else
        return V_hat * m.chain(x)
    end

end

In [26]:
@load "emulator_1.bson" model

In [35]:
function get_eigenglaciers(omegas, F, q)
    
    F_mean = sum(F .* omegas, dims=2);
    F_bar = F .- F_mean;

    Z = diagm(sqrt.(omegas[1, :] * n_grid_points))
    U, S, V = tsvd(Z * transpose(F_bar), q);
    lamda = S.^2 / n_grid_points
    V_hat = V * diagm(sqrt.(lamda));
    
    return V_hat, F_bar, F_mean
end;

In [45]:
q = 50
F = log10.(Data)
F = replace!(F, -Inf=>0)

dirichlet_dist = Dirichlet(n_samples, 1)

model_index = 1
omegas = transpose(rand(dirichlet_dist, 1))
omegas_0 = omegas ./ size(omegas)[1];
    
V_hat, F_bar, F_mean = get_eigenglaciers(omegas, F, q);

In [46]:
alpha_b = 3;
beta_b = 3;
beta_dist = Beta(alpha_b, beta_b);
X_prior = rand(beta_dist, n_parameters, 100000);
X_0 = mean(X_prior, dims=2);

In [47]:
Y_target = log10.(Obs);
Y_target = replace!(Y_target, -Inf=>0);

In [48]:
grid_resolution = ones(n_grid_points) .* 9000
sigma = 10
rho = 1.0 / (1e4 .^ 2)
point_area = (grid_resolution) .^ 2
K = point_area .* rho
sigma_hat = sqrt.(sigma .^ 2 ./ K .^ 2)

X_min = minimum(X_scaled, dims=2);
X_max = maximum(X_scaled, dims=2);

In [49]:
using TransformVariables, TransformedLogDensities, LogDensityProblems, LogDensityProblemsAD,
    DynamicHMC, DynamicHMC.Diagnostics, Parameters, Statistics, Random

In [191]:
struct SampleBayesProblem
    nn
    X_min::AbstractArray
    X_max::AbstractArray
    Y_target::AbstractArray
    sigma_hat::AbstractArray
    nu::Int
    alpha::Float16
end

In [236]:
function (problem::SampleBayesProblem)(θ)
    @unpack α = θ               # extract the parameters
    @unpack nn, X_min, X_max, Y_target, sigma_hat, nu, alpha = problem       # extract the data
    
    Y_pred = 10 .^ nn(α, true)
    r = Y_pred .- Y_target
    t = r ./ sigma_hat

    # StudentT distribution
    loglikelihood = sum(
        loggamma((nu + 1) / 2)
        - loggamma(nu / 2)
        .- log.(sqrt.(pi * nu) .* sigma_hat)
        .- (nu + 1) / 2.0 * log.(1 .+ 1.0 / nu .* t .^ 2)
    )
    # Beta prior
    X_bar = (α .- X_min) ./ (X_max - X_min)
    logprior = sum(
        (alpha_b - 1) * log.(X_bar) + (beta_b - 1) * log.(1 .- X_bar) 
        .+ loggamma(alpha_b + beta_b) 
        .- loggamma(alpha_b)
        .- loggamma(beta_b)
    )
    (alpha * loglikelihood + logprior)
end

In [237]:
nu = 1
alpha = 0.01
logp = SampleBayesProblem(model, X_min, X_max, Y_target, sigma_hat, nu, alpha)
logp((α = X_0,)) # make sure that it works

-1370.248332616955

In [242]:
trans = as((α = as(Array, as𝕀, n_parameters),))
P = TransformedLogDensity(trans, logp)
∇P = ADgradient(:Zygote, P)

Zygote AD wrapper for TransformedLogDensity of dimension 8

In [244]:
results = mcmc_with_warmup(Random.default_rng(), ∇P, 2; 
    initialization = (q = vec(X_0), ))

┌ Info: found initial stepsize
└   ϵ = 0.502
┌ Info: Starting MCMC
│   total_steps = 75
└   tuning = "stepsize"
┌ Info: MCMC progress
│   step = 1
│   seconds_per_step = 0.027
│   estimated_seconds_left = 2.0
└   ϵ = 0.502


LoadError: InterruptException:

In [14]:
summarize_tree_statistics(results.tree_statistics)

LoadError: UndefVarError: results not defined

In [16]:
posterior = transform.(trans, eachcol(results.posterior_matrix))
posterior_α = first.(posterior)
mean(posterior_α)

LoadError: UndefVarError: results not defined

In [246]:
logp_p(x) = LogDensityProblems.logdensity(P, x)
∇logp_p(x) = LogDensityProblems.logdensity_and_gradient(∇P, x)[2]
result_pf = pathfinder(logp_p, ∇logp_p; dim=8)

┌ Warning: 5 (33.3%) updates to the inverse Hessian estimate were rejected to keep it positive definite.
└ @ Pathfinder ~/.julia/packages/Pathfinder/ZV8MJ/src/singlepath.jl:216


Single-path Pathfinder result
  tries: 1
  draws: 5
  fit iteration: 3 (total: 14)
  fit ELBO: -1371.45 ± 0.51
  fit distribution: MvNormal{Float64, Pathfinder.WoodburyPDMat{Float64, Diagonal{Float64, Vector{Float64}}, Matrix{Float64}, Matrix{Float64}, Pathfinder.WoodburyPDFactorization{Float64, Diagonal{Float64, Vector{Float64}}, LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}, UpperTriangular{Float64, Matrix{Float64}}}}, Vector{Float64}}(
dim: 8
μ: [-1.0813196977168364, 0.37126320473869334, -1.1849660359519867, 0.617318696822632, -0.2605616707657209, 0.8880730024998509, 0.26528739046198485, -1.605444709739312]
Σ: [1.3516982683983274 -0.44533806713299645 … -0.3050134611113764 0.1370283712889094; -0.44533806713299645 0.9040749637270831 … 0.216734523049508 -0.09492744687601606; … ; -0.3050134611113764 0.216734523049508 … 0.5718372866231103 -0.07182006806986431; 0.1370283712889094 -0.09492744687601606 … -0.07182006806986431 0.33530408954617047]
)


In [ ]:
init_params = result_pf.draws[:, 1]
result_dhmc1 = mcmc_with_warmup(
    Random.GLOBAL_RNG,
    ∇P,
    10;
    initialization=(; q=init_params),
    reporter=NoProgressReport(),
)

In [118]:
Pkg.add("Optim")
using Optim

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [429524aa] + Optim v1.7.4
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [163]:
Pkg.add("StatsBase")
using StatsBase

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [2913bbd2] + StatsBase v0.33.21
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [149]:
?HMC

search: HMC HMCDA SGHMC DynamicHMC eachmatch Hypergeometric



```
HMC(ϵ::Float64, n_leapfrog::Int)
```

Hamiltonian Monte Carlo sampler with static trajectory.

Arguments:

  * `ϵ::Float64` : The leapfrog step size to use.
  * `n_leapfrog::Int` : The number of leapfrog steps to use.

Usage:

```julia
HMC(0.05, 10)
```

Tips:

  * If you are receiving gradient errors when using `HMC`, try reducing the leapfrog step size `ϵ`, e.g.

```julia
# Original step size
sample(gdemo([1.5, 2]), HMC(0.1, 10), 1000)

# Reduced step size
sample(gdemo([1.5, 2]), HMC(0.01, 10), 1000)
```


In [124]:
tdist = TDist(nu)
rand(tdist, 1000)

1000-element Vector{Float64}:
   1.963095806225691
 -11.191839136683544
  -0.5405180190802729
   0.6754577528075394
   1.223256402817003
   0.06478276005202036
   0.4067019823503065
  -2.9572871790650477
   2.0459740837569935
   0.9584262297970466
   0.5348255362067189
   3.6399824310350817
  -3.302387517366774
   ⋮
   2.945932632535606
  -0.5223813025118502
  -0.38266098471397475
  24.001574437840446
  -0.42581545950653066
  -2.076358294464149
   1.3742437673227927
  -0.9366917561343404
  -0.5768313165031164
  -0.0058989115121783845
  -0.2484742324077987
   2.6743543860772565

8×1 Matrix{Float64}:
 0.6484512218503838
 0.647865052318695
 0.6269898233495614
 0.6481996742741711
 0.650645159308533
 0.6353300690661433
 0.6486107349450604
 0.6381724595183763

In [217]:
?min

search: min minmax minimum minimum! minorticks_on minorticks_off Fminbox X_min



```
min(x, y, ...)
```

Return the minimum of the arguments (with respect to [`isless`](@ref)). See also the [`minimum`](@ref) function to take the minimum element from a collection.

# Examples

```jldoctest
julia> min(2, 5, 1)
1
```
